In [ ]:
!pip install numpy pandas scikit-learn librosa

In [ ]:
!pip install mrmr-selection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
import mrmr
import librosa
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

# Make sure the uploaded file is named "kaggle.json"
if 'kaggle.json' in uploaded:
    print('kaggle.json uploaded!')
else:
    print('Please upload the kaggle.json file.')

Please upload the kaggle.json file.


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"yvanmeli","key":"bab9cb9542f5271b1a2af70138f98c28"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# list datasets on kaggle with the name Ravdess
!kaggle datasets list -s TESS

ref                                                                title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ejlok1/toronto-emotional-speech-set-tess                           Toronto emotional speech set (TESS)                 428MB  2019-08-24 23:03:36          17975        151  0.875            
dmitrybabko/speech-emotion-recognition-en                          Speech Emotion Recognition (en)                     987MB  2021-01-25 12:59:50           8238         89  0.875            
uldisvalainis/audio-emotions                                       Audio emotions                                        1GB  2020-06-09 12:56:17           2064         33  0.75             
cracc97/features                             

In [ ]:
# Authenticate into Kaggle and download the required dataset then unzip it in the folder in which we are found

import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('ejlok1/toronto-emotional-speech-set-tess', path='.', unzip=True)


#Data Processing


*   Audio Normalization
*   Silence Removal




In [ ]:
def normalize(X):
    max_X = np.max(np.abs(X))
    Y = X / max_X

    return Y

In [ ]:
def remove_silence(X, factor):
    max_X = np.max(np.abs(X))
    decision_threshold = max_X / factor

    # Find the indices of samples above the decision threshold
    indices_useful_X, _ = find_peaks(np.abs(X), height=decision_threshold)

    # Extract the useful samples
    Y = X[indices_useful_X[0]:indices_useful_X[-1] + 1]

    return Y


In [ ]:
import os
import librosa
import soundfile as sf

def normalise_remove_silence(input_filename, output_directory, factor):
    # Read the input audio file
    x, fs = librosa.load(input_filename, sr=None)

    # Normalize the audio using librosa
    y = librosa.util.normalize(x)

    # Remove silence using librosa.effects.trim
    z, _ = librosa.effects.trim(y, top_db=factor)

    # Extract the filename and extension
    filename, extension = os.path.splitext(os.path.basename(input_filename))

    # Create the output filename in the specified directory
    output_filename = os.path.join(output_directory, f"{filename}_N_RS{factor}{extension}")

    # Write the processed audio to the output file
    sf.write(output_filename, z, fs)

# Audio Feature Extraction with MRMR

In [ ]:
from scipy.stats import mode

def global_feature_computation(feature_matrix, computations):
    computed_features = []
    for computation in computations:
        if computation == "mean":
            computed_features.append(np.mean(feature_matrix, axis=1))
        elif computation == "min":
            computed_features.append(np.min(feature_matrix, axis=1))
        elif computation == "max":
            computed_features.append(np.max(feature_matrix, axis=1))
        elif computation == "std":
            computed_features.append(np.std(feature_matrix, axis=1))
        elif computation == "range":
            computed_features.append(np.ptp(feature_matrix, axis=1))
        elif computation == "mode":
            # Use scipy.stats.mode to get mode and count
            mode_result = mode(feature_matrix, axis=1)
            computed_features.append(mode_result.mode.flatten())
        elif computation == "median":
            computed_features.append(np.median(feature_matrix, axis=1))
        elif computation == "1st_quartile":
            computed_features.append(np.percentile(feature_matrix, 25, axis=1))
        elif computation == "3rd_quartile":
            computed_features.append(np.percentile(feature_matrix, 75, axis=1))
        # Add conditions for other computations

    return np.concatenate(computed_features)

In [ ]:
# extract features available in the librosa library --> list not exhaustiv

def extract_features(audio_path, features, global_computation):
    # Load the normalized and silence-removed audio
    audio, _ = librosa.load(audio_path, sr=None)

    # Placeholder for feature extraction (replace with actual implementation)
    feature_list = []

    # Extract selected features
    for feature_name in features:
        if feature_name == "spectral_flatness":
            spectral_flatness = librosa.feature.spectral_flatness(y=audio)
            feature_list.append(global_feature_computation(spectral_flatness, global_computation))
        elif feature_name == "spectral_centroid":
            spectral_centroid = librosa.feature.spectral_centroid(y=audio)
            feature_list.append(global_feature_computation(spectral_centroid, global_computation))
        elif feature_name == "mfcc":
            mfcc = librosa.feature.mfcc(y=audio)
            feature_list.append(global_feature_computation(mfcc, global_computation))
        #elif feature_name == "delta":
            #mfcc_delta = librosa.feature.delta(mfcc)
            #feature_list.append(global_feature_computation(mfcc_delta, global_computation))
        elif feature_name == "melspectrogram":
            mel_spectrum = librosa.feature.melspectrogram(y=audio)
            feature_list.append(global_feature_computation(mel_spectrum, global_computation))

    # Combine the extracted features into a single feature vector
    feature_vector = np.concatenate(feature_list)

    return feature_vector


In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

#model training



*   SVM



In [ ]:
features = ["spectral_flatness", "spectral_centroid", "mfcc", "melspectrogram"]
#global_computation = ["mean", "min", "max"]
global_computation = ["mean", "min", "max", "std", "range", "mode", "median"]

class_names = ["Neutral", "Cal", "Happiness", "Sadness", "Angry", "Fear", "Disgust", "Surprise"]

factor = 200

num_selected_features = 5

###Normalizing and removing the silence

In [ ]:
# Set the base directory where actor directories are located
base_directory = '/content/TESS Toronto emotional speech set data'
output_directory = '/content/processed_data'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# First loop: Normalize and remove silence
for actor_directory in os.listdir(base_directory):
    actor_path = os.path.join(base_directory, actor_directory)

    # Check if it's a directory
    if os.path.isdir(actor_path):

        # Iterate over audio files in the actor's directory
        for audio_file in os.listdir(actor_path):
            audio_path = os.path.join(actor_path, audio_file)

            # Normalize and remove silence
            normalise_remove_silence(audio_path, output_directory, factor)


In [ ]:
# List to store the paths of normalized audio files
normalized_audio_paths = []

for audio_file in os.listdir(output_directory):
    normalized_audio_path = os.path.join(output_directory, audio_file)

    # Append to the list of normalized audio paths
    normalized_audio_paths.append(normalized_audio_path)


In [ ]:
normalized_audio_paths

['/content/processed_data/OAF_dime_disgust_N_RS200.wav',
 '/content/processed_data/YAF_peg_sad_N_RS200.wav',
 '/content/processed_data/OAF_chalk_angry_N_RS200.wav',
 '/content/processed_data/OAF_sell_ps_N_RS200.wav',
 '/content/processed_data/OAF_dab_happy_N_RS200.wav',
 '/content/processed_data/OAF_pain_disgust_N_RS200.wav',
 '/content/processed_data/OAF_choice_happy_N_RS200.wav',
 '/content/processed_data/OAF_should_happy_N_RS200.wav',
 '/content/processed_data/OAF_long_neutral_N_RS200.wav',
 '/content/processed_data/YAF_far_angry_N_RS200.wav',
 '/content/processed_data/YAF_cheek_disgust_N_RS200.wav',
 '/content/processed_data/OAF_vine_happy_N_RS200.wav',
 '/content/processed_data/OAF_germ_neutral_N_RS200.wav',
 '/content/processed_data/YAF_cab_sad_N_RS200.wav',
 '/content/processed_data/OAF_note_sad_N_RS200.wav',
 '/content/processed_data/YAF_chair_neutral_N_RS200.wav',
 '/content/processed_data/YAF_base_fear_N_RS200.wav',
 '/content/processed_data/OAF_void_neutral_N_RS200.wav',
 '/

In [ ]:
# Initialize lists for audio files, labels, and features
features_list = []
audio_files = []

for normalized_audio_path in normalized_audio_paths:
    extracted_features = extract_features(normalized_audio_path, features, global_computation)

    # Append to lists or perform further processing as needed
    audio_files.append(normalized_audio_path)
    features_list.append(extracted_features)

# Convert features_list to a NumPy array if needed
features_list = np.array(features_list)

In [ ]:
features_list.shape

(2800, 1050)